- make a histogram of the lepton pt as a regular variable,  the "cut" as a string category, the Z boson pt as another regular variable and finally the deltaR(Z boson, lepton) as another regular variable
- save each of the selection cuts currently applied now as different masks
- for each mask save the candidate lepton pT distribution in the histogram, save the cut in the string category, and save the Z boson pt, and the deltaR(z boson, candidate lepton)
- for the Z boson pT you can do something similar to our gen matching i.e. find the Z boson gen particle in the event

In [1]:
import json
import uproot
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from coffea import nanoevents
from coffea import processor
import time

import argparse
import warnings
import pyarrow as pa
import pyarrow.parquet as pq
import pickle as pkl
import pandas as pd
import os
import sys
sys.path.append("../")

In [2]:
from collections import defaultdict
import pickle as pkl
import pyarrow as pa
import awkward as ak
import numpy as np
import pandas as pd
import json
import os
import shutil
import pathlib
from typing import List, Optional
import pyarrow.parquet as pq

from coffea import processor
from coffea.nanoevents.methods import candidate, vector
from coffea.analysis_tools import Weights, PackedSelection
from boostedhiggs.utils import match_HWW
from boostedhiggs.btag import btagWPs
from boostedhiggs.btag import BTagCorrector

import warnings
warnings.filterwarnings("ignore", message="Found duplicate branch ")
warnings.filterwarnings("ignore", category=DeprecationWarning)
np.seterr(invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [101]:
# define the configuration and processor
fileset = {}
fileset["DYJetsToLL_Pt-100To250"] = [f"../DY.root"]

job_name = '/0-1'
channels=['ele', 'mu', 'had']

from boostedhiggs.hwwprocessor_nocuts import HwwProcessor_nocuts
p = HwwProcessor_nocuts(year='2017', channels=channels, output_location='./outfiles' + job_name)

In [102]:
# remove output directory to store fresh samples
if os.path.exists('./outfiles'):
    shutil.rmtree('./outfiles')

In [103]:
executor = processor.IterativeExecutor(status=True)
run = processor.Runner(
    executor=executor, savemetrics=True, schema=nanoevents.NanoAODSchema, chunksize=10000
)
    
out, metrics = run(
fileset, "Events", processor_instance=p
)

# dump to pickle
filehandler = open('./outfiles/' + job_name + '.pkl', "wb")
pkl.dump(out, filehandler)
filehandler.close()

# merge parquet
for ch in channels:
    data = pd.read_parquet('./outfiles/' + job_name + ch + '/parquet')
    data.to_parquet('./outfiles/' + job_name + '_' + ch + '.parquet')

    # remove old parquet files
    os.system('rm -rf ./outfiles/' + job_name + ch)

Processing:   0%|          | 0/6 [00:00<?, ?chunk/s]

/opt/anaconda3/envs/coffea-env/lib/python3.7/site-packages/awkward/_connect/_numpy.py:210: RuntimeWarning: divide by zero encountered in log
  *[nplike.asarray(x) for x in inputs], **kwargs


In [104]:
data = {}
for ch in channels:  
    data[ch] = pq.read_table('./outfiles/' + job_name + '_' + ch + '.parquet').to_pandas()

In [105]:
data['ele'].keys()

Index(['lep_pt', 'lep_isolation', 'lep_misolation', 'lep_fj_m',
       'lep_fj_bjets_ophem', 'lep_fj_dr', 'fj_msoftdrop', 'fj_pt',
       'lep_met_mt', 'met', 'ht', 'weight', 'trigger_iso', 'trigger_noiso',
       'metfilters', 'leptonKin', 'oneMuon', 'oneElectron', 'notaus_mu',
       'notaus_ele', 'MuonIsolation', 'ElectronIsolation', 'leptonInJet',
       'anti_bjettag', 'mt_lep_met', 'oneFatjet', 'fatjetKin',
       'fatjetSoftdrop', 'qcdrho'],
      dtype='object')

## Make histograms:

In [12]:
import pickle as pkl
import pyarrow.parquet as pq
import pyarrow as pa
import awkward as ak
import numpy as np
import pandas as pd
import json
import os
import sys
import glob
import shutil
import pathlib
from typing import List, Optional

import argparse
from coffea import processor
from coffea.nanoevents.methods import candidate, vector
from coffea.analysis_tools import Weights, PackedSelection

import hist as hist2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import mplhep as hep
from hist.intervals import clopper_pearson_interval

import warnings
warnings.filterwarnings("ignore", message="Found duplicate branch ")

In [93]:
# make the cuts
cuts = {}

cuts['ele'] = {'leptonKin': data['ele']['leptonKin']>40, 
               'oneElectron': data['ele']['oneElectron'],
               'notaus_ele': data['ele']['notaus_ele'],
               'ElectronIsolation': data['ele']['ElectronIsolation']==1,               
               'leptonInJet': data['ele']['leptonInJet']==1,
               'anti_bjettag': data['ele']['anti_bjettag']==1,
               'ht': data['ele']['ht'],
               'mt_lep_met': data['ele']['mt_lep_met']==1,               
              }

cuts['mu'] = {'leptonKin': data['mu']['leptonKin']>30, 
               'oneMuon': data['mu']['oneMuon'],
               'notaus_mu': data['mu']['notaus_mu'],
               'MuonIsolation': data['mu']['MuonIsolation']==1,                             
               'leptonInJet': data['mu']['leptonInJet']==1,
               'anti_bjettag': data['mu']['anti_bjettag']==1,
               'ht': data['mu']['ht'],
               'mt_lep_met': data['mu']['mt_lep_met']==1,               
              }

cuts['had'] = {'oneFatjet': data['had']['oneFatjet'], 
               'fatjetKin': data['had']['fatjetKin']>300,
               'fatjetSoftdrop': data['had']['fatjetSoftdrop']==1,
               'qcdrho': data['had']['qcdrho']==1,                             
               'met': data['had']['met']<200,            
              }

In [118]:
hists = {}
year = '2017'
channels = ['ele', 'mu']

hists[year] = {}
for ch in channels:

    hists[year][ch] = hist2.Hist(
        hist2.axis.Regular(50,0, 250, name='lepton_pt', label='lepton_pt', flow=False),
        hist2.axis.StrCategory([], name='cuts', growth=True)
    )

# loop over the cuts and fill the histograms
for ch in channels:
    for key, value in cuts[ch].items():
        hists[year][ch].fill(
            data[ch][value]['lep_pt'], key,
        )

In [124]:
odir = './plots3'
# remove output directory to store fresh samples
if os.path.exists(f'{odir}'):
    shutil.rmtree(f'{odir}')
for ch in channels:
    for cut in hists[year][ch].axes[-1]:
        fig, ax = plt.subplots(figsize=(8, 5))
        hep.histplot(hists[year][ch][{'cuts': cut}], ax=ax)
        ax.set_xlabel(f"{'lep_pt'}")
        ax.set_title(f'{ch} channel after \n {cut} cut')
        hep.cms.lumitext(f"{year} (13 TeV)", ax=ax)
        hep.cms.text("Work in Progress", ax=ax)

        if not os.path.exists(f'{odir}/'):
            os.makedirs(f'{odir}/')

        plt.savefig(f'{odir}/{ch}_{cut}.pdf')
        plt.close()

In [123]:
odir = './plots2'
# remove output directory to store fresh samples
if os.path.exists(f'{odir}'):
    shutil.rmtree(f'{odir}')
for ch in channels:
        fig, ax = plt.subplots(figsize=(8, 5))
        for cut in hists[year][ch].axes[-1]:        
            hep.histplot(hists[year][ch][{'cuts': cut}], ax=ax, label=cut)
        
        ax.set_xlabel(f"{'lep_pt'}")
        ax.set_title(f'{ch} channel after \n {cut} cut')
        hep.cms.lumitext(f"{year} (13 TeV)", ax=ax)
        hep.cms.text("Work in Progress", ax=ax)

        if not os.path.exists(f'{odir}/'):
            os.makedirs(f'{odir}/')
        ax.legend()
        plt.savefig(f'{odir}/{ch}_{cut}.pdf')
        plt.close()